# 0_ Initialisation de l'environnement colab.

In [ ]:
! pip install transformers

     |████████████████████████████████| 2.9 MB 7.2 MB/s 
     |████████████████████████████████| 52 kB 2.0 MB/s 
     |████████████████████████████████| 895 kB 57.8 MB/s 
     |████████████████████████████████| 636 kB 71.9 MB/s 
     |████████████████████████████████| 3.3 MB 60.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from google.colab import drive
drive.mount('/drive')


Mounted at /drive


In [ ]:
drive_dir = '/drive/MyDrive'
data_dir = drive_dir + '/nlp_nature'

# 1_ Chargement des libraries

In [ ]:
import pandas as pd
import numpy as np

import torch

# 2_ Load data.

In [ ]:
conclusion_df=pd.read_csv(data_dir + '/jco_conclusion_df_bevacizumab_29092021.csv')

In [ ]:
conclusion_df.head()

,Unnamed: 0,link,title,journal,authors,type,field,article,conclusion
0,0,https://ascopubs.org/doi/abs/10.1200/JCO.20.01225,\n\n \n \n\n ...,Journal of Clinical Oncology,"Chiara Cremolini,Search for articles by this a...",ORIGINAL REPORTS,Gastrointestinal Cancer,\n##Abstract##PURPOSE##A proper estimation of ...,FOLFOXIRI + bevacizumab significantly and mean...
1,1,https://ascopubs.org/doi/abs/10.1200/JOP.2015....,\n\n \n \n\n ...,Journal of Oncology Practice,"Stacie B. Dusetzina,Search for articles by thi...",Health Policy,Original Contributions,\n##Abstract##Section:##Choose##Top of page##A...,FOLFOXIRI + bevacizumab significantly and mean...
2,2,https://ascopubs.org/doi/abs/10.1200/JCO.2015....,\n\n \n \n\n ...,Journal of Clinical Oncology,"Bradley J. Monk,Search for articles by this au...",ORIGINAL REPORTS,Gynecologic Cancer,\n##Abstract##Section:##Choose##Top of page##A...,"On the basis of the PFS, overall response rate..."
3,3,https://ascopubs.org/doi/abs/10.1200/JCO.19.03158,\n\n \n \n\n ...,Journal of Clinical Oncology,"Martin Reck,Search for articles by this author...",ORIGINAL REPORTS,Thoracic Oncology,\n##Abstract##Section:##Choose##Top of page##A...,ABCP seems tolerable and manageable versus ACP...
4,4,https://ascopubs.org/doi/abs/10.1200/JCO.2014....,\n\n \n \n\n ...,Journal of Clinical Oncology,"Miguel Martín,Search for articles by this auth...",ORIGINAL REPORTS,Breast Cancer,\n##Abstract##Section:##Choose##Top of page##A...,The addition of bevacizumab to ET in first-l...


# 3_ Load the transformers



La différence avec l'autre analyse est que l'on utilise la GPU pour booster le traitement

In [ ]:
# Test the GPU
torch.cuda.is_available()

True

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
from transformers import AutoTokenizer,  AutoModelForSequenceClassification


model_name_0 = "siebert/sentiment-roberta-large-english"
model_name_1 = "ProsusAI/finbert"

l_classifier_attributes = [{'name':'siebert', 'sentiments':['positive', 'negative']}, {'name':'finbert', 'sentiments':['positive', 'negative', 'neutral']}]
l_tokenizer = [AutoTokenizer.from_pretrained(model_name_0), AutoTokenizer.from_pretrained(model_name_1)]
l_classifiers = [AutoModelForSequenceClassification.from_pretrained(model_name_0), AutoModelForSequenceClassification.from_pretrained(model_name_1)]


Downloading:   0%|          | 0.00/256 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/687 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/780k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/758 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/418M [00:00<?, ?B/s]

In [ ]:
l_conclusion=conclusion_df['conclusion'].to_list()


# 4_ Classification des conclusions

On porte les tensors et les modèles sur gpu avant de lancer l'analyse. C'est gourmand en RAM, on se limite à des batch de 4 sentences mais les performances sont bluffantes, on gagne 2 ordres de grandeur

In [ ]:
def assign_device(token_outputs) :
   #print(token_outputs)
   token_tensor = token_outputs['input_ids'].to(device)
   attention_mask = token_outputs['attention_mask'].to(device)

   output = {'input_ids' : token_tensor, 
          'attention_mask' : attention_mask}

   return output

In [ ]:

from tqdm import tqdm
import time

batch_size = 4
nb_batch = len(l_conclusion) // batch_size 


start_0=time.time()

for i, attr in enumerate(l_classifier_attributes) : 

  name = attr['name']
  columns = attr['sentiments']

  #Passage du modele sur la GPU
  classifier_device = l_classifiers[i].to(device)
  #Transform the logits to probabilities
  softmax=torch.nn.Softmax(dim=1).to(device)
  
  tokenizer = l_tokenizer[i]

  for nb in tqdm(range(nb_batch+1)) :

     l_run = l_conclusion[nb*batch_size: min([len(l_conclusion),(nb+1)*batch_size])]

     # Tokenize sentence list 
     input_tokens = assign_device(tokenizer(l_run , truncation=True, padding=True, return_tensors="pt"))

     # Model apply to get the missing logits
     outputs = classifier_device(**input_tokens)

     #Transform the logits tensor to numpy after going down to the cpu
     if nb == 0 :
       logits = outputs.logits.cpu().detach().numpy()
       probs=softmax(outputs.logits).cpu().detach().numpy()
     else :
       logits = np.concatenate((logits,outputs.logits.cpu().detach().numpy()), axis=0)
       probs = np.concatenate((probs,softmax(outputs.logits).cpu().detach().numpy()), axis=0)

  #Add logits to the dataframe
  conclusion_df[[f'{name}_logit_{c}' for c in columns]] = logits
  
  #Add probs to the dataframe
  conclusion_df[[f'{name}_prob_{c}' for c in columns]] = probs


end_0=time.time()

print(f'duree {end_0 - start_0}')

100%|██████████| 114/114 [00:01<00:00, 75.09it/s]

duree 6.240475416183472


In [ ]:
conclusion_df.head()

,Unnamed: 0,link,title,journal,authors,type,field,article,conclusion,siebert_logit_positive,siebert_logit_negative,siebert_prob_positive,siebert_prob_negative,finbert_logit_positive,finbert_logit_negative,finbert_logit_neutral,finbert_prob_positive,finbert_prob_negative,finbert_prob_neutral
0,0,https://ascopubs.org/doi/abs/10.1200/JCO.20.01225,\n\n \n \n\n ...,Journal of Clinical Oncology,"Chiara Cremolini,Search for articles by this a...",ORIGINAL REPORTS,Gastrointestinal Cancer,\n##Abstract##PURPOSE##A proper estimation of ...,FOLFOXIRI + bevacizumab significantly and mean...,-3.670126,2.828857,0.001503,0.998497,2.136958,-2.068751,-1.257303,0.953766,0.014221,0.032013
1,1,https://ascopubs.org/doi/abs/10.1200/JOP.2015....,\n\n \n \n\n ...,Journal of Oncology Practice,"Stacie B. Dusetzina,Search for articles by thi...",Health Policy,Original Contributions,\n##Abstract##Section:##Choose##Top of page##A...,FOLFOXIRI + bevacizumab significantly and mean...,-3.670126,2.828857,0.001503,0.998497,2.136958,-2.068751,-1.257303,0.953766,0.014221,0.032013
2,2,https://ascopubs.org/doi/abs/10.1200/JCO.2015....,\n\n \n \n\n ...,Journal of Clinical Oncology,"Bradley J. Monk,Search for articles by this au...",ORIGINAL REPORTS,Gynecologic Cancer,\n##Abstract##Section:##Choose##Top of page##A...,"On the basis of the PFS, overall response rate...",-3.201620,2.425135,0.003587,0.996413,1.283581,-1.626127,0.241374,0.710648,0.038724,0.250628
3,3,https://ascopubs.org/doi/abs/10.1200/JCO.19.03158,\n\n \n \n\n ...,Journal of Clinical Oncology,"Martin Reck,Search for articles by this author...",ORIGINAL REPORTS,Thoracic Oncology,\n##Abstract##Section:##Choose##Top of page##A...,ABCP seems tolerable and manageable versus ACP...,-3.513865,2.686230,0.002025,0.997975,-0.052418,-0.800742,0.857222,0.252743,0.119588,0.627669
4,4,https://ascopubs.org/doi/abs/10.1200/JCO.2014....,\n\n \n \n\n ...,Journal of Clinical Oncology,"Miguel Martín,Search for articles by this auth...",ORIGINAL REPORTS,Breast Cancer,\n##Abstract##Section:##Choose##Top of page##A...,The addition of bevacizumab to ET in first-l...,3.866877,-3.450049,0.999336,0.000664,1.311700,-0.260613,-1.670056,0.794743,0.164960,0.040296


In [ ]:
conclusion_df.to_csv(data_dir + '/jco_scores_gpu_df_bevacizumab_30092021.csv')